In [ ]:
import mlflow
import optuna
import pandas as pd
import category_encoders as ce

# Preprocessamento
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from feature_engine.selection import DropConstantFeatures, DropCorrelatedFeatures
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder, PowerTransformer, PolynomialFeatures

# Modelos
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

# Metricas
from sklearn.metrics import roc_auc_score, confusion_matrix


# Exploração dos dados

Nesta sessão, irei explorar os dados para entender melhor o que está por trás e
decidir como a parte de experimentação será realizada. 

## Impressões iniciais

Nesta etapa irei observar os dados de forma superficial para entender como estão
estruturados e quais informações estão disponíveis.

In [ ]:
# importando os dados
dados = pd.read_csv('../data/raw/customer_booking.csv', encoding='latin-1')

In [ ]:
# Checando como os dados estão
dados.head()

In [ ]:
# Verificando o tamanho do dataset
dados.shape

In [ ]:
# Verificando a info sobre os dados
dados.info()

In [ ]:
# Verificando se há valores duplicados
dados.drop_duplicates().shape

## Considerações a partir da análise inicial

A partir das informações iniciais, podemos entender que temos um dataset com 50 
mil registros e 14 colunas, onde tais registros não estão atribuídos a um 
usuário em específico. Devido a essa condição, podemos observar dados duplicados.

Além disso, podemos perceber que o tipo de dado da maior parte das colunas está 
coerente com o seu conteúdo, com exceção das colunas **wants_extra_baggage**, 
**wants_preferred_seat** e **wants_in_flight_meals**, que são colunas 
categóricas que foram previamente codificadas. 

Por último, também é possível perceber que não existem valores ausentes.

O nosso target está na coluna **booking complete**.

## Aplicando correções iniciais

Nesta etapa, irei apenas excluir os dados duplicados. Também será realizada a 
correção no tipo dos dados.

In [ ]:
# Dropando dados duplicados
dados = dados.drop_duplicates()

In [ ]:
# Alterando o tipo de dado da coluna 'wants_extra_baggage'
dados['wants_extra_baggage'] = dados['wants_extra_baggage'].astype('object')

In [ ]:
# Alterando o tipo de dado da coluna 'wants_preferred_seat'
dados['wants_preferred_seat'] = dados['wants_preferred_seat'].astype('object')

In [ ]:
# Alterando o tipo de dado da coluna 'wants_in_flight_meals'
dados['wants_in_flight_meals'] = dados['wants_in_flight_meals'].astype('object')

## Explorando as variáveis categóricas
Nesta etapa, irei verificar se existem valores raros em cada coluna. Caso 
existam, será necessário uni-los em uma única categoria, pois valores raros
podem compremeter o treinamento do modelo.

In [ ]:
# Obtendo as colunas categóricas
cat_cols = dados.select_dtypes(include='object').columns.tolist()

In [ ]:
# Verificando as colunas categóricas
dados.loc[:, cat_cols].head()

In [ ]:
# Verificando se existem registros raros na coluna 'sales_channel'
dados['sales_channel'].value_counts()

In [ ]:
# Verificando se existem registros raros na coluna 'trip_type'
dados['trip_type'].value_counts()

In [ ]:
# Verificando se existem registros raros na coluna 'flight_day'
dados['flight_day'].value_counts()


In [ ]:
# Verificando se existem registros raros na coluna 'route'
dados['route'].value_counts()

In [ ]:
# Verificando se existem registros raros na coluna 'booking_origin'
dados['booking_origin'].value_counts()

In [ ]:
# Verificando se existem registros raros na coluna 'wants_extra_baggage'
dados['wants_extra_baggage'].value_counts()

In [ ]:
# Verificando se existem registros raros na coluna 'wants_preferred_seat'
dados['wants_preferred_seat'].value_counts()

In [ ]:
# Verificando se existem registros raros na coluna 'wants_in_flight_meals'
dados['wants_in_flight_meals'].value_counts()

## Considerações a partir da análise das variáveis categóricas
A partir das 6 colunas categóricas, podemos observar que existem valores raros
em duas delas: **route** e **booking_origin**. Portanto, irei agrupar os valores
raros em uma nova categoria. Tais categorias serão agrupadas em uma única de nome
**'others'**. Além disso, ainda é possível observar que essas features possuem 
uma alta cardinalidade, o que pode ser um problema para o treinamento do modelo.

## Corrigindo as variáveis categóricas que apresentaram problemas
Nesta fase, os dados com menos de 4 registros serão agrupados em uma nova 
categoria.

### Booking origin

In [ ]:
# Buscando os registros que atendem ao critério de raridade
country_booking_less_than_four = dados['booking_origin'].value_counts() < 8

In [ ]:
# Salvando em uma lista os paises que atendem ao critério de raridade
countries = dados['booking_origin'].value_counts()[country_booking_less_than_four].index.to_list()

In [ ]:
# Criando função para substituir os registros raros por 'other'
func_rare_registry = lambda country: 'other' if country in countries else country

In [ ]:
# Aplicando a função
dados['booking_origin'] = dados['booking_origin'].apply(func_rare_registry)

### Route

In [ ]:
# Buscando os registros que atendem ao critério de raridade
route_less_than_four = dados['route'].value_counts() < 8

In [ ]:
# Salvando em uma lista os paises que atendem ao critério de raridade
routes = dados['route'].value_counts()[route_less_than_four].index.to_list()

In [ ]:
# Criando função para substituir os registros raros por 'other'
func_rare_registry = lambda route: 'other' if route in routes else route

In [ ]:
# Aplicando a função
dados['route'] = dados['route'].apply(func_rare_registry)

## Explorando as variáveis numéricas
Na segunda parte, irei observar as variáveis numéricas para entender a 
variabilidade dos dados e se existem outliers.


In [ ]:
# Obtendo as colunas numéricas
num_cols = dados.select_dtypes(['int', 'float']).columns.tolist()

In [ ]:
# Verificando os dados numéricos
dados.loc[:, num_cols].head()

In [ ]:
# Verificando a distribuição dos dados da coluna 'num_passengers'
dados['num_passengers'].plot(kind='box')

In [ ]:
# Verificando a distribuição dos dados da coluna 'purchase_lead'
dados['purchase_lead'].plot(kind='box')

In [ ]:
# Verificando a distribuição dos dados da coluna 'length_of_stay'
dados['length_of_stay'].plot(kind='box')

In [ ]:
# Verificando a distribuição dos dados da coluna 'flight_hour'
dados['flight_hour'].plot(kind='box')

In [ ]:
# Verificando a distribuição dos dados da coluna 'flight_duration'
dados['flight_duration'].plot(kind='box')

## Considerações a partir da análise das variáveis numéricas
A partir das colunas examinadas, podemos observar que existem outliers em três
delas. Entretanto, tais outliers aparentam ser naturais, e não provenientes de
erros de digitação ou de coleta de dados. Portanto, não serão tratados.

Além disso, os dados apresentam uma variabilidade considerável, fazendo com
não seja necessário realizar nenhum tipo de exclusão de coluna.



## Explorando o target
E por último, o target será analisado para verificar se há desbalanceamento.

In [ ]:
dados['booking_complete'].value_counts()

## Considerações a partir da análise do target

A partir da análise do target, podemos observar que o dataset está desbalanceado.

## Criando novas features

Nesta fase, irei criar novas features a partir das variáveis existentes.

In [ ]:
# Criando função para determinar em que parte do dia o voo ocorre
func_time_of_day = lambda hour: 'morning' if hour >= 6 and hour < 12 else ('afternoon' if hour >= 12 and hour < 18 else 'night')

# Aplicando a função
dados['time_of_day'] = dados['flight_hour'].apply(func_time_of_day)

In [ ]:
# Calculando a hora de chegada
dados['arrival_hour'] = dados['flight_hour'] + dados['flight_duration']

In [ ]:
# Criando função para verificar se o voo chega no mesmo dia
func_same_day = lambda hour: 'yes' if hour < 24 else 'no'

# Aplicando a função
dados['same_day_arrival'] = dados['arrival_hour'].apply(func_same_day)

In [ ]:
# Criando função para checar se a viagem vai ocorrer no final de semana
func_weekend = lambda day: 'yes' if day == 'Sat' or day == 'Sun' else 'no'

# Aplicando a função
dados['weekend_trip'] = dados['flight_day'].apply(func_weekend)

In [ ]:
# Calculando o tempo de estadia em meses
dados['length_of_stay_months'] = dados['length_of_stay'] / 30

# calculando o tempo de estadia em semanas
dados['length_of_stay_years'] = dados['length_of_stay'] / 360

# Experimentação
Aqui, irei realizar a experimentação para encontrar o melhor modelo para o
problema em questão. 

Inicialmente, irei testar um modelo de classificação que é sensível a
escala dos dados: a **Logistic Regression**. Em seguida, irei incluir 
os modelos baseados em árvore: **Decision Tree**, **Random Forest**, 
**AdaBoost** e **LightGBM**.

Como métrica de avaliação, irei usar a **AUC**. Além disso, irei usar a
Cross Validation para avaliar a performance dos modelos, registrando o 
desempenho de cada fold e a média final.

Para acessar os experimentos, digite o comando abaixo no terminal:

```mlflow ui```

In [ ]:
# Define o local para salvar os exoerimentos
mlflow.set_tracking_uri('../mlruns')

# Criando/acessando o experimento
mlflow.set_experiment('Comparando modelos')

In [ ]:
# Dividindo os dados em variáveis dependentes e independentes
x = dados.drop(columns='booking_complete')
y = dados['booking_complete']

# Dividindo os dados em treino e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x,
                                                        y,
                                                        test_size=0.15,
                                                        random_state=200)

# Dividindo os dados em dev e teste
x_dev, x_teste, y_dev, y_teste = train_test_split(x_teste,
                                                  y_teste,
                                                  test_size=0.50,
                                                  random_state=200)

In [ ]:
# Obtendo as colunas com alta dimensionalidade
high_dim_cols = ['route', 'booking_origin']

# Obtendo as colunas categóricas
cat_cols = x.select_dtypes(include='object').columns.tolist()
cat_cols = [col for col in cat_cols if col not in high_dim_cols]

# Obtendo as colunas numéricas
num_cols = x.select_dtypes(['int', 'float']).columns.tolist()

# Instanciando um KFold Estratificado
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=200) 

In [ ]:
# Criando dicionário com os modelos
dict_models_scale_sensitive = {"LR": LogisticRegression(random_state=200,
                                                        class_weight='balanced')}

dict_models_tree_based = {"LGBM": LGBMClassifier(is_unbalance=True,
                                                 random_state=200),
                          "DT": DecisionTreeClassifier(class_weight='balanced',
                                                       random_state=200),
                          "RF": RandomForestClassifier(class_weight='balanced',
                                                       random_state=200),
                          "ADA": AdaBoostClassifier(DecisionTreeClassifier(class_weight='balanced',
                                                                           random_state=200))}

# Criando dicionário com os encoders
dict_encoders = {"OHE": OneHotEncoder(drop='first'),
                 "TE": ce.TargetEncoder(),
                 "BE": ce.BinaryEncoder(),
                 "ME": ce.MEstimateEncoder(),
                 "WOE": ce.WOEEncoder(),
                 "CE": ce.CatBoostEncoder(),
                 "GE":ce.GrayEncoder()}

dict_scalers = {"SS": StandardScaler(),
                "RS": RobustScaler()}

# Criando dicionário com os transformers
dict_transformers = {"PT": PowerTransformer(),
                     "PF": PolynomialFeatures()}

In [ ]:
# Iniciando os experimentos sem transformers
for tag, model in dict_models_scale_sensitive.items():
    for tag_encoder, encoder in dict_encoders.items():
        for tag_scaler, scaler in dict_scalers.items():
            
            # Gerando a tag de identificação do modelo
            nome_modelo = f'{tag}_{tag_encoder}_{tag_scaler}'
            
            with mlflow.start_run(run_name=nome_modelo):
                 
                 # Criando os pipeline com os transformers
                 pipe_cat = Pipeline([('encoder', encoder)])
                 pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
                 pipe_num = Pipeline([('scaler', scaler)])
                 
                 # Criando o transformador
                 transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
                                                 ('num', pipe_num, num_cols),
                                                 ('high_dim', pipe_high_dim, high_dim_cols)])
                 
                 # Criando o pipeline final
                 pipe = Pipeline([('transformer', transformer),
                                  ('remove_corr', DropCorrelatedFeatures()),
                                  ('remove_const', DropConstantFeatures()),
                                 ('model', model)])
                 
                 # Executando o cross validation
                 cross_val_scores = cross_val_score(pipe, x_treino, y_treino, cv=kf, scoring='roc_auc')
                 
                 # Calculando a média das métricas
                 mean_score = cross_val_scores.mean()
                 
                 # Salvando a tag de identificação
                 mlflow.set_tag('modelo', tag)               
                 
                 # Salvando a métrica da folder 1
                 mlflow.log_metric('roc_auc_fold_1', cross_val_scores[0])
                 
                 # Salvando a métrica da folder 2
                 mlflow.log_metric('roc_auc_fold_2', cross_val_scores[1])
                
                 # Salvando a métrica da folder 3
                 mlflow.log_metric('roc_auc_fold_3', cross_val_scores[2])
                
                 # Salvando a métrica da folder 4
                 mlflow.log_metric('roc_auc_fold_4', cross_val_scores[3])
                
                 # Salvando a métrica da folder 5
                 mlflow.log_metric('roc_auc_fold_5', cross_val_scores[4])
                 
                 # Salvando as métricas
                 mlflow.log_metric('roc_auc_mean', mean_score)

In [ ]:
# Iniciando os experimentos com transformers
for tag, model in dict_models_scale_sensitive.items():
    for tag_encoder, encoder in dict_encoders.items():
        for tag_scaler, scaler in dict_scalers.items():
            for tag_transformer, transformer in dict_transformers.items():
            
                # Gerando a tag de identificação do modelo
                nome_modelo = f'{tag}_{tag_encoder}_{tag_scaler}_{tag_transformer}'

                with mlflow.start_run(run_name=nome_modelo):

                     # Criando os pipeline com os transformers
                     pipe_cat = Pipeline([('encoder', encoder)])
                     pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
                     pipe_num = Pipeline([('scaler', scaler),
                                          ('transformer', transformer)])

                     # Criando o transformador
                     transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
                                                     ('num', pipe_num, num_cols),
                                                     ('high_dim', pipe_high_dim, high_dim_cols)])

                     # Criando o pipeline final
                     pipe = Pipeline([('transformer', transformer),
                                     ('remove_corr', DropCorrelatedFeatures()),
                                     ('remove_const', DropConstantFeatures()),
                                     ('model', model)])

                     # Executando o cross validation
                     cross_val_scores = cross_val_score(pipe, x_treino, y_treino, cv=kf, scoring='roc_auc')

                     # Calculando a média das métricas
                     mean_score = cross_val_scores.mean()

                     # Salvando a tag de identificação
                     mlflow.set_tag('modelo', tag)               

                     # Salvando a métrica da folder 1
                     mlflow.log_metric('roc_auc_fold_1', cross_val_scores[0])

                     # Salvando a métrica da folder 2
                     mlflow.log_metric('roc_auc_fold_2', cross_val_scores[1])

                     # Salvando a métrica da folder 3
                     mlflow.log_metric('roc_auc_fold_3', cross_val_scores[2])

                     # Salvando a métrica da folder 4
                     mlflow.log_metric('roc_auc_fold_4', cross_val_scores[3])

                     # Salvando a métrica da folder 5
                     mlflow.log_metric('roc_auc_fold_5', cross_val_scores[4])

                     # Salvando as métricas
                     mlflow.log_metric('roc_auc_mean', mean_score)

In [ ]:
# Iniciando os experimentos sem transformers
for tag, model in dict_models_tree_based.items():
    for tag_encoder, encoder in dict_encoders.items():
            
            # Gerando a tag de identificação do modelo
            nome_modelo = f'{tag}_{tag_encoder}'
            
            with mlflow.start_run(run_name=nome_modelo):
                 
                 # Criando os pipeline com os transformers
                 pipe_cat = Pipeline([('encoder', encoder)])
                 pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
                 
                 # Criando o transformador
                 transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
                                                 ('high_dim', pipe_high_dim, high_dim_cols)])
                 
                 # Criando o pipeline final
                 pipe = Pipeline([('transformer', transformer),
                                 ('remove_corr', DropCorrelatedFeatures()),
                                 ('remove_const', DropConstantFeatures()),
                                 ('model', model)])
                 
                 # Executando o cross validation
                 cross_val_scores = cross_val_score(pipe, x_treino, y_treino, cv=kf, scoring='roc_auc')
                 
                 # Calculando a média das métricas
                 mean_score = cross_val_scores.mean()
                 
                 # Salvando a tag de identificação
                 mlflow.set_tag('modelo', tag)               
                 
                 # Salvando a métrica da folder 1
                 mlflow.log_metric('roc_auc_fold_1', cross_val_scores[0])
                 
                 # Salvando a métrica da folder 2
                 mlflow.log_metric('roc_auc_fold_2', cross_val_scores[1])
                
                 # Salvando a métrica da folder 3
                 mlflow.log_metric('roc_auc_fold_3', cross_val_scores[2])
                
                 # Salvando a métrica da folder 4
                 mlflow.log_metric('roc_auc_fold_4', cross_val_scores[3])
                
                 # Salvando a métrica da folder 5
                 mlflow.log_metric('roc_auc_fold_5', cross_val_scores[4])
                 
                 # Salvando as métricas
                 mlflow.log_metric('roc_auc_mean', mean_score)

In [49]:
# Salvando os resultados
resultados_experimentos = mlflow.search_runs()

In [53]:
# Ordenando as colunas pela métrica
colunas = ['tags.mlflow.runName', 'metrics.roc_auc_fold_1', 
           'metrics.roc_auc_fold_2', 'metrics.roc_auc_fold_3', 
           'metrics.roc_auc_fold_4', 'metrics.roc_auc_fold_5', 
           'metrics.roc_auc_mean']

resultados_experimentos.sort_values('metrics.roc_auc_mean', ascending=False).loc[:, colunas]

,tags.mlflow.runName,metrics.roc_auc_fold_1,metrics.roc_auc_fold_2,metrics.roc_auc_fold_3,metrics.roc_auc_fold_4,metrics.roc_auc_fold_5,metrics.roc_auc_mean
23,LGBM_WOE,0.795743,0.772043,0.769088,0.767051,0.769244,0.774634
25,LGBM_BE,0.795251,0.771549,0.769222,0.766216,0.768250,0.774097
26,LGBM_TE,0.795743,0.772043,0.769088,0.764865,0.767550,0.773858
24,LGBM_ME,0.795743,0.772043,0.769088,0.764865,0.767550,0.773858
21,LGBM_GE,0.793527,0.771631,0.769129,0.766161,0.768527,0.773795
...,...,...,...,...,...,...,...
17,DT_ME,0.600777,0.589602,0.599350,0.596365,0.601854,0.597590
16,DT_WOE,0.600777,0.589602,0.599350,0.595267,0.598996,0.596799
20,DT_OHE,0.596324,0.587790,0.598400,0.594546,0.601752,0.595762
15,DT_CE,0.574956,0.567625,0.569634,0.562030,0.568083,0.568465
